# Training 

In [ ]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
%matplotlib inline# interactive plotting in  jupyter-notebook
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense #fully connected layer
from keras.layers import LSTM
from keras.callbacks import TensorBoard  # for plotting computational graph

### Loading Dataset

In [ ]:
# load dataset
dataset = read_csv('root.csv', parse_dates=['Date'], index_col='Date')
dataset = dataset[['Flow','TimePeriod','AverageJT','AverageSpeed','DataQuality','LinkLength','day']]
dataset.dropna(inplace=True,axis=0,how='any')
values = dataset.values
dataset.head()

### Windowing  Method to create 96 sample time series as each sample
Our each sample contains 7 variable  for single instance 
but we need time series in single sample so we need to concatinate sevral time sample one after one 
for that we are usng Windowing method


In [ ]:

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


### Data type Equlizaltion  and  Normalization (using scikit learn package)
    values = values.astype('float32')
This commond converts  all data is in float32 format including string 
if any alphabetical string is found in dataset which can not be converted to float like("Naan","Na") in that case this commond will give an error


    scaler = MinMaxScaler(feature_range=(0, 1))
This commond will create scale-variable "scaler"  for normalizing all dataset 

    scaled = scaler.fit_transform(values)
Here We are Normlizing dataset  usinng previously created scale variable





In [ ]:
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled.shape

In [ ]:

# specify the number of lag hours
n_hours = 96 #Days is used to Decide input steps it should be decided on input we have 96 input for a day
n_features = 7 #
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)


# split into train and test sets
values = reframed.values
n_train_hours = 365 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
#split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)



### Building model and Training 
We are using Single LSTM Layer here
and once Fully Connected layer over here

We are  using ADAM optimizer 

Calculating  Mean absolute error to check loss after each itration 

based upon Mean absolute error(mae) we are adjusting weights and learning rate 

After each itration we are calculating following matrics

Itration :-50
mini batch size :-72
          (96*31)/minibatch size 

In [ ]:

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam',metrics=['mae','mse'])
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
# fit network
#history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=1, shuffle=False,callbacks=[tensorboard])

### model Save

In [ ]:
model.save("model.h5")